# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [3]:
# Drop na values
spaceship = spaceship.dropna()

In [4]:
# Clean 'Cabin' column
spaceship['Cabin'] = spaceship['Cabin'].str[0]

In [5]:
# Drop PassengerId and Name
spaceship.drop(columns=['PassengerId', 'Name'], inplace=True)

In [6]:
# Converting categorical columns to dummies
spaceship = df = pd.get_dummies(spaceship, columns=['HomePlanet', 'Cabin', 'Destination'])

In [7]:
# Splitting the data into features (X) and target variable (y)
X = spaceship.drop(columns=['Transported'])
y = spaceship['Transported']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)

In [8]:
# Scaling the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Fit the scaler only on the training data
X_train_scaled = scaler.fit_transform(X_train)

# Use the same scaler to transform the test data
X_test_scaled = scaler.transform(X_test)

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [9]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [10]:
# bagging classifier
bagging_clf = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),  # Base estimator (e.g., decision tree)
    n_estimators=100,  # Number of base estimators
    max_samples=0.8,  # Proportion of training samples to use for each base estimator
    bootstrap=True,   # Whether to use bootstrap sampling (True for bagging)
    random_state=22   # Random seed for reproducibility
)

* Evaluate your model

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Fit the BaggingClassifier to the training data
bagging_clf.fit(X_train, y_train)

# Make predictions on the test data
pred = bagging_clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, pred)

# Calculate precision
precision = precision_score(y_test, pred)

# Calculate recall
recall = recall_score(y_test, pred)

# Calculate F1-score
f1 = f1_score(y_test, pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

c:\Users\david\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Accuracy: 0.8048411497730711
Precision: 0.8304821150855366
Recall: 0.7818448023426061
F1-score: 0.8054298642533937


For this lab we will use Grid Search.

In [14]:
from sklearn.model_selection import GridSearchCV

# Define a smaller parameter grid
param_grid = {
    'base_estimator__max_depth': [None, 10, 20],
    'base_estimator__min_samples_split': [2, 5],
    'base_estimator__min_samples_leaf': [1, 2],
    'base_estimator__max_features': [None, 'sqrt'],
    'n_estimators': [50, 100],
    'max_samples': [0.5, 0.8],
    'bootstrap': [True, False],
    'bootstrap_features': [True, False]
}


- Run Grid Search

In [15]:
# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=bagging_clf,
    param_grid=param_grid,
    cv=3,  # Use 3-fold cross-validation
    n_jobs=-1,  # Utilize all available CPU cores
    scoring='accuracy'
)

# Fit GridSearchCV to the data
grid_search.fit(X_train, y_train)

# Best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

c:\Users\david\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Best Parameters: {'base_estimator__max_depth': 10, 'base_estimator__max_features': 'sqrt', 'base_estimator__min_samples_leaf': 2, 'base_estimator__min_samples_split': 2, 'bootstrap': False, 'bootstrap_features': False, 'max_samples': 0.8, 'n_estimators': 100}
Best Score: 0.7978774786365278


- Evaluate your model

In [16]:
# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
test_pred = best_model.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, test_pred))
print("Test Precision:", precision_score(y_test, test_pred))
print("Test Recall:", recall_score(y_test, test_pred))
print("Test F1-score:", f1_score(y_test, test_pred))

Test Accuracy: 0.8071104387291982
Test Precision: 0.8184523809523809
Test Recall: 0.8052708638360175
Test F1-score: 0.8118081180811808


In [17]:
# Now lets try RandomSearch
from sklearn.model_selection import RandomizedSearchCV

# Define the parameter distribution
param_dist = {
    'base_estimator__max_depth': [None, 10, 20],
    'base_estimator__min_samples_split': [2, 5],
    'base_estimator__min_samples_leaf': [1, 2],
    'base_estimator__max_features': [None, 'sqrt'],
    'n_estimators': [50, 100],
    'max_samples': [0.5, 0.8],
    'bootstrap': [True, False],
    'bootstrap_features': [True, False]
}

In [18]:
# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=bagging_clf,
    param_distributions=param_dist,
    n_iter=20,  # Limit to 20 iterations
    cv=3,  # Use 3-fold cross-validation
    n_jobs=-1,  # Utilize all available CPU cores
    scoring='accuracy',
    random_state=22
)

# Fit RandomizedSearchCV to the data
random_search.fit(X_train, y_train)

# Best parameters and best score
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

c:\Users\david\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Best Parameters: {'n_estimators': 50, 'max_samples': 0.8, 'bootstrap_features': False, 'bootstrap': True, 'base_estimator__min_samples_split': 5, 'base_estimator__min_samples_leaf': 2, 'base_estimator__max_features': 'sqrt', 'base_estimator__max_depth': 10}
Best Score: 0.796741975148695


In [19]:
# Evaluate the best model on the test set
best_model = random_search.best_estimator_
test_pred = best_model.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, test_pred))
print("Test Precision:", precision_score(y_test, test_pred))
print("Test Recall:", recall_score(y_test, test_pred))
print("Test F1-score:", f1_score(y_test, test_pred))

Test Accuracy: 0.8055975794251135
Test Precision: 0.8207831325301205
Test Recall: 0.7979502196193266
Test F1-score: 0.8092056421677802


##  Conclusions

Accuracy:

Original: 0.8048
GridSearch: 0.8071 (Slight improvement)
RandomSearch: 0.8056 (Slight improvement)
Interpretation: Both GridSearchCV and RandomizedSearchCV provide a small improvement in accuracy over the original model.

Precision:

Original: 0.8305
GridSearch: 0.8185 (Slight decrease)
RandomSearch: 0.8208 (Slight decrease)
Interpretation: Both hyperparameter tuning methods result in a slight decrease in precision, indicating a slight increase in false positives.

Recall:

Original: 0.7818
GridSearch: 0.8053 (Improvement)
RandomSearch: 0.7980 (Improvement)
Interpretation: Both methods improve recall significantly, indicating the model captures more true positives with fewer false negatives.

F1-score:

Original: 0.8054
GridSearch: 0.8118 (Improvement)
RandomSearch: 0.8092 (Improvement)
Interpretation: Both methods result in a better F1-score, with GridSearchCV performing slightly better than RandomizedSearchCV. This shows a balanced improvement in both precision and recall.

Conclusion

Hyperparameter Tuning Effectiveness:

Both GridSearchCV and RandomizedSearchCV improve the overall performance of the BaggingClassifier compared to the original model.
GridSearchCV provides a slightly better balance, achieving the highest recall and F1-score among the three models.
RandomizedSearchCV offers improvements in accuracy and recall, with a minor trade-off in precision compared to the original model.